In [ ]:
#WE DONT'T NEED TO USE ALL OF THIS ANYMORE
# Install required packages
!pip uninstall kafka-python -y
!pip install kafka-python-ng pandas

In [ ]:
from time import sleep
from json import dumps
from kafka import KafkaProducer
import pandas as pd

topic_name = 'bigdata'
kafka_server = 'localhost:9092'

print(f"Connecting to Kafka at {kafka_server}...")
producer = KafkaProducer(
    bootstrap_servers=kafka_server,
    value_serializer=lambda x: dumps(x).encode('utf-8')
)
print("Connected to Kafka successfully!")

file_path = '../dataset/weather_dataset.csv'
print(f"\nReading data from: {file_path}")
csv_data = pd.read_csv(file_path)
print(f"Loaded {len(csv_data)} records")

# OPTION 1: Send only first 100 records (for testing)
# Uncomment this line to test with small dataset:
# csv_data = csv_data.head(100)

# OPTION 2: Send all records but FAST (0.1s delay)
# Change sleep(5) to sleep(0.1) below for faster processing

print("\n" + "="*60)
print(f"Starting to send {len(csv_data)} records to Kafka...")
print("="*60)

for index, row in csv_data.iterrows():
    data = row.to_dict()
    
    for key, value in data.items():
        if not isinstance(value, str):
            data[key] = str(value)

    # Send the data as a Kafka message
    producer.send(topic_name, value=data)
    
    # Print progress every 10 records
    if (index + 1) % 10 == 0 or index == 0:
        print(f"[{index + 1}/{len(csv_data)}] Sent: Date={data.get('date_time', 'N/A')}, "
              f"Weather={data.get('weather', 'N/A')}, Temp={data.get('tempC', 'N/A')}°C")
    
    # CHANGE THIS FOR SPEED:
    # sleep(5)    # Original: 5 seconds (188 hours for all data!)
    sleep(0.1)    # Fast: 0.1 seconds (~3.8 hours for all data)
    # sleep(0)    # Fastest: No delay (~few minutes for all data)

producer.close()
print("\n" + "="*60)
print("All data sent successfully!")
print("="*60)


In [ ]:
from time import sleep
from json import dumps
from kafka import KafkaProducer
import pandas as pd

topic_name = 'bigdata'
kafka_server = 'localhost:9092'

print(f"QUICK TEST MODE - Sending first 50 records only")
print(f"Connecting to Kafka at {kafka_server}...")
producer = KafkaProducer(
    bootstrap_servers=kafka_server,
    value_serializer=lambda x: dumps(x).encode('utf-8')
)
print("Connected!")

file_path = '../dataset/weather_dataset.csv'
csv_data = pd.read_csv(file_path).head(50)  # Only first 50 records
print(f"Loaded {len(csv_data)} records for testing\n")

sent_count = 0
for index, row in csv_data.iterrows():
    data = row.to_dict()
    
    # Convert all values to strings
    for key, value in data.items():
        if not isinstance(value, str):
            data[key] = str(value)
    
    # Send to Kafka
    producer.send(topic_name, value=data)
    sent_count += 1
    
    print(f"Sent {sent_count}/50: {data.get('date_time', 'N/A')}, Weather={data.get('weather', 'N/A')}")
    
    # No delay for quick testing!

producer.close()
print(f"\nSUCCESS! Sent {sent_count} records to Kafka")
print("Check http://localhost:5000 now - you should see predictions!")
